<a href="https://colab.research.google.com/github/aneesAyy/hello-world/blob/master/dip_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import random
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import math
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from sklearn.metrics import f1_score

In [ ]:
training_file_path = '/content/drive/MyDrive/Colab Notebooks/Segmentation/train_images.npy'
training_labels_path = '/content/drive/MyDrive/Colab Notebooks/Segmentation/training_labels_path.npy'
testing_file_path = '/content/drive/MyDrive/Colab Notebooks/Segmentation/test_images.npy'
testing_labels_path = '/content/drive/MyDrive/Colab Notebooks/Segmentation/testing_labels_path.npy'

training_images, training_masks, testing_images, testing_masks = np.load(training_file_path), np.load(training_labels_path), np.load(testing_file_path), np.load(testing_labels_path) 

print('Shape of training images and training lables is: ', training_images.shape, ',', training_masks.shape)
print('Shape of testing images and testing lables is: ', testing_images.shape, ',', testing_masks.shape)

Shape of training images and training lables is:  (762, 256, 256, 3) , (762, 256, 256)
Shape of testing images and testing lables is:  (76, 256, 256, 3) , (76, 256, 256)


In [ ]:
# https://idiotdeveloper.com/unet-implementation-in-tensorflow-using-keras-api/
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 16)
    s2, p2 = encoder_block(p1, 32)
    s3, p3 = encoder_block(p2, 64)
    s4, p4 = encoder_block(p3, 128)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 128)
    d2 = decoder_block(d1, s3, 64)
    d3 = decoder_block(d2, s2, 32)
    d4 = decoder_block(d3, s1, 16)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

model = build_unet((256, 256, 3))
model.summary()


Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 256, 256, 16  448         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 256, 256, 16  64         ['conv2d_38[0][0]']              
 ormalization)                  )                                                             

In [ ]:



# metrics = [dice_coef, iou, Recall(), Precision()]
my_optimiser = Adam(lr=0.00001)
model.compile(loss='BinaryCrossentropy', optimizer=my_optimiser, metrics='accuracy')
epochs = 50

print('Training Models')

my_model_history = model.fit(
        x=training_images, y=training_masks, epochs=epochs, validation_split = 0.1)



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training...
Epoch 1/50
22/22 [==============================] - 30s 509ms/step - loss: -58.0250 - accuracy: 0.1400 - val_loss: -11.6114 - val_accuracy: 0.1745
Epoch 2/50
22/22 [==============================] - 8s 361ms/step - loss: -74.4157 - accuracy: 0.1673 - val_loss: -6.9461 - val_accuracy: 0.1251
Epoch 3/50
22/22 [==============================] - 8s 365ms/step - loss: -91.6743 - accuracy: 0.1956 - val_loss: -7.4131 - val_accuracy: 0.1414
Epoch 4/50
22/22 [==============================] - 8s 368ms/step - loss: -108.9823 - accuracy: 0.2278 - val_loss: -9.4133 - val_accuracy: 0.1665
Epoch 5/50
22/22 [==============================] - 8s 369ms/step - loss: -125.3673 - accuracy: 0.2597 - val_loss: -12.1942 - val_accuracy: 0.2038
Epoch 6/50
22/22 [==============================] - 8s 367ms/step - loss: -140.5677 - accuracy: 0.2832 - val_loss: -16.1568 - val_accuracy: 0.2615
Epoch 7/50
22/22 [==============================] - 8s 364ms/step - loss: -155.2950 - accuracy: 0.3039 - val_lo

NameError: ignored